# Read in in LA-COMPASS 2D data

In [ ]:
!cd ~/CODES/pylacompass && git pull

In [ ]:
from pathlib import Path

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

import astropy.units as u
import astropy.constants as c

from scipy.interpolate import griddata

from pylacompass import read_data, read_hdf5_file, twod_plot, read_torqfile, convert_to_cgs, get_snapshot_numbers

au = c.au.cgs.value
yr = (u.yr).to(u.s)

%matplotlib inline
plt.style.use({'figure.dpi': 150})

## Reading data

Set filename of the `hdf5` file

In [ ]:
fname = 'data.hdf5'

In [ ]:
# dirname = '/e/ldata/users/birnstiel/lanl_data_new/2018/'
dirname = '/e/ldata/users/drazkowska/LA-COMPASS_planet'

This reads in the specified directory from a sub-directory `bin_data`. The simulation parameters are parsed from the file given as `inputfile`. If keyword `fname` is given, we store the result in a hdf5 file with that name.

In [ ]:
d = read_data(directory=dirname, inputfile='planet2D.input.new', n=-1, fname=fname)

In [ ]:
get_snapshot_numbers(fname)

Once the hdf5 file exists, we can also read from it. By default that just opens the file and the data is read when needed. If everything is to be read at once into memory, pass the `lowmem=False` keyword.

In [ ]:
d, f5  = read_hdf5_file(fname, n=134, lowmem=False)

Convert to cgs units

In [ ]:
dcgs = convert_to_cgs(d)

Also read the torq file

In [ ]:
dt = read_torqfile(d, Path(dirname) / 'torq1d.dat')

# Find largest particle size

**Note:** some of the methods used here require `d.sigma_d` to be a numpy array. If you used the `lowmem` keyword of `read_hdf5_file`, then you need to replace `d.sigma_d` with a 

In [ ]:
# define a boolean condition to determine whether the bin is considered filled or empty

cond = d.sigma_d / (np.sum(d.sigma_d,-1)[:, :, None] + 1e-45) > 1e-10

# find the last filled index (argmax finds the first maximum, so we invert the array)

max_index = cond.shape[-1] - cond[:, :, ::-1].argmax(-1) - 1

a_max = d.a[max_index]

### Close the hdf5-file

In [ ]:
f5.close()

## Artistic gas & dust plot

In [ ]:
sig_large = dcgs.sigma_d[..., dcgs.a>0.1].sum(-1)
sig_small = dcgs.sigma_d[..., dcgs.a<1e-3].sum(-1)
sig_g = dcgs.sigma_g.copy()

# fix the phi boundary

sig_g[:, -1] = sig_g[:, -2]
sig_large[:, -1] = sig_large[:, -2]
sig_small[:, -1] = sig_small[:, -2]

In [ ]:
from astropy.visualization import SqrtStretch
from astropy.visualization import AsinhStretch
from astropy.visualization import LogStretch
from astropy.visualization import LinearStretch
from astropy.visualization import PowerStretch
from astropy.visualization import ImageNormalize

In [ ]:
# calculate the cartesian positions of the regular r/phi grid

xx, yy = np.meshgrid(dcgs.x, dcgs.y)
xy1 = xx * np.cos(yy)
xy2 = xx * np.sin(yy)

In [ ]:
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator

# make a regular cartesian grid for interpolating an image

x = np.linspace(-dcgs.x[-1], dcgs.x[-1], int(np.ceil((2 * dcgs.x[-1]) / (dcgs.x[1] - dcgs.x[0]))))
X,Y = np.meshgrid(x, x)

points = np.array([xy1, xy2]).reshape([2, -1]).T
newpoints = np.array([X, Y]).reshape([2, -1]).T

values_g = sig_g.T.ravel()
values_lg = sig_large.T.ravel()
values_sm = sig_small.T.ravel()

# this does what griddata does, just in a way that the grid can be re-used
tri = Delaunay(points)

# Perform the interpolation with the given values:
interpolator = LinearNDInterpolator(tri, values_g)
newvalues_g = interpolator(newpoints).reshape(X.shape)

interpolator = LinearNDInterpolator(tri, values_sm)
newvalues_sm = interpolator(newpoints).reshape(X.shape)

interpolator = LinearNDInterpolator(tri, values_lg)
newvalues_lg = interpolator(newpoints).reshape(X.shape)

# newvalues_g  = griddata(points, values_g, newpoints).reshape(X.shape)
# newvalues_lg = griddata(points, values_lg, newpoints).reshape(X.shape)
# newvalues_sm = griddata(points, values_sm, newpoints).reshape(X.shape)

Gas image

In [ ]:
normg = ImageNormalize(vmin=1e-1, vmax=1e2, stretch=PowerStretch(0.3), clip=True)

cmap='Greys_r'

f, ax = plt.subplots(figsize=(3, 3))
ax.imshow(newvalues_g, extent=[-x[-1] / au, x[-1] / au, -x[-1] / au, x[-1] / au], cmap=cmap, norm=normg, origin='lower')

xmax = np.sin(np.pi/4) * (dcgs.x[-1] / au)
ax.set_xlim(-xmax, xmax)
ax.set_ylim(-xmax, xmax)
ax.set_xlabel('x [au]')
ax.set_ylabel('y [au]');

Define a dust norms

In [ ]:
norm_sm = ImageNormalize(vmin=1e-1, vmax=1e1, stretch=PowerStretch(0.3), clip=True)
norm_lg = ImageNormalize(vmin=1e-1, vmax=1e1, stretch=PowerStretch(0.3), clip=True)

In [ ]:
norm_sm = ImageNormalize(vmin=5e-2, vmax=1e1, stretch=AsinhStretch(0.01), clip=True)
norm_lg = ImageNormalize(vmin=5e-2, vmax=1e1, stretch=AsinhStretch(0.01), clip=True)

Large dust image

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(6, 3))
axs[0].imshow(newvalues_sm, extent=[-x[-1] / au, x[-1] / au, -x[-1] / au, x[-1] / au], cmap=cmap, norm=norm_sm)
axs[1].imshow(newvalues_lg, extent=[-x[-1] / au, x[-1] / au, -x[-1] / au, x[-1] / au], cmap=cmap, norm=norm_lg)

xmax = np.sin(np.pi/4) * (dcgs.x[-1] / au)

for ax, title in zip(axs, ['small', 'large']):
    ax.set_title(title + ' dust', fontsize='small')
    ax.set_xlim(-xmax, xmax)
    ax.set_ylim(-xmax, xmax)
    ax.set_xlabel('x [au]')
    ax.set_ylabel('y [au]')

Define colormaps

In [ ]:
cmap_b = LinearSegmentedColormap.from_list('mycmap', ['#0044FF00', '#0088FFFF'])
sns.palplot(cmap_b(np.linspace(0, 1, 10)), size=0.3)

cmap_r = LinearSegmentedColormap.from_list('mycmap', ['#FF220000', '#FF4400FF'])
sns.palplot(cmap_r(np.linspace(0, 1, 10)), size=0.3)

In [ ]:
extent = [-x[-1] / au, x[-1] / au, -x[-1] / au, x[-1] / au]
xmax = np.sin(np.pi/4) * (dcgs.x[-1] / au)

f, ax = plt.subplots()
im1 = ax.imshow(newvalues_g, cmap='Greys', extent=extent, norm=normg)
im3 = ax.imshow(newvalues_lg, cmap=cmap_b, extent=extent, norm=norm_lg)
im2 = ax.imshow(newvalues_sm, cmap=cmap_r, extent=extent, norm=norm_sm)

pos = ax.get_position()

# make gas colorbar

cax1 = f.add_axes([pos.x1, pos.y0, pos.height/20, pos.height])
cb1 = f.colorbar(
    im1,
    cax=cax1,
    ticks=[1e-2, 1e-1, 1e0, 1e1, 1e2]
)
cb1.ax.set_ylabel('$\Sigma_\mathsf{g}$ [g/cm$^2$]')

# make small dust colorbar

cpos = cb1.ax.get_position()
cax2 = f.add_axes([pos.x0, pos.y1 + 2 * cpos.width, pos.width, cpos.width])
cb2 = f.colorbar(
    im2,
    cax=cax2,
    ticks=[1e-2, 1e-1, 1e0, 1e1, 1e2],
    orientation='horizontal',
    drawedges=False,
)

# make large dust colorbar

cpos = cb2.ax.get_position()
cax3 = f.add_axes([cpos.x0, cpos.y1, cpos.width, cpos.height])
cb3 = f.colorbar(
    im3,
    cax=cax3,
    ticks=[1e-2, 1e-1, 1e0, 1e1, 1e2],
    orientation='horizontal',
    drawedges=False,
)
cb3.ax.xaxis.set_ticks_position('top')
cb3.ax.xaxis.set_label_position('top') 

# set other dust colorbar properties

for cb, title in zip([cb2, cb3],['small', 'large']):
    cb.ax.tick_params(labelsize='small')
    cb.ax.text(
        1.01, 0.5,
        '$\Sigma_\mathsf{d, '+title+'}$ [g/cm$^2$]',
        fontsize='x-small', rotation=0,
        horizontalalignment='left', verticalalignment='center',
        transform=cb.ax.transAxes
    )
    cb.solids.set_edgecolor('face')
    cb.solids.set_rasterized(True)
    
# mask inner region
ax.add_artist(plt.Circle((0, 0), dcgs.x[0] / au, color='w'))


# limits, labels, annotations

ax.set_xlim(-xmax, xmax)
ax.set_ylim(-xmax, xmax)
ax.set_xlabel('x [au]')
ax.set_ylabel('y [au]')

# planet annotation

xp, yp = 3, -9.3
ax.plot(xp, yp,'k+')
ax.annotate(r'1 $M_\mathrm{Jup}$ planet', xy=(xp, yp*1.03), fontsize='x-small', xycoords='data',
            xytext=(xp, -22), textcoords='data',
            arrowprops=dict(facecolor='black', arrowstyle='->'),
            horizontalalignment='center', verticalalignment='top', 
            )

# vortex annotation

xv, yv = -15, -1.5
ax.annotate(r'vortex', xy=(1.03 * xv, yv), fontsize='x-small', xycoords='data',
            xytext=(0.03, 0.03), textcoords='axes fraction',
            arrowprops=dict(
                facecolor='black',
                arrowstyle='->',
                connectionstyle='angle'
            ),
            horizontalalignment='left', verticalalignment='bottom', 
            )

# pressure bump annotation

xb, yb = 0, 18
ax.annotate(r'pressure bump', xy=(xb, yb), fontsize='x-small', xycoords='data',
            xytext=(-0.95 * xmax, yb), textcoords='data',
            arrowprops=dict(
                facecolor='black',
                arrowstyle='->',
                connectionstyle='angle'
            ),
            horizontalalignment='left', verticalalignment='center', 
            )


f.savefig('image.pdf', bbox_inches='tight', transparent=True)
f.savefig('image.png', bbox_inches='tight', transparent=True, dpi=370)